In [ ]:
from Bio.Blast import NCBIWWW

In [ ]:
sequence_data = open("blast_example.fasta").read() 
result_handle = NCBIWWW.qblast("blastn", "nt", sequence_data)

In [ ]:
blastn -db alun -query search.fsa -out results.xml -outfmt 5

In [ ]:
from Bio.Blast.Applications import NcbiblastnCommandline 
blastn_cline = NcbiblastnCommandline(query = "search.fsa", db = "local_alu_db\local_alu_db", outfmt = 5, out = "results.xml") 
stdout, stderr = blastn_cline()
print(stdout)
print(stderr)

In [ ]:
from Bio import SeqIO 
from Bio.Blast import NCBIWWW
seq_record = next(SeqIO.parse(open('blast_input.fasta'),'fasta')) 
print(seq_record.id)
result_handle = NCBIWWW.qblast("blastn", "nt", seq_record.seq)

In [ ]:
with open('results.xml', 'w') as save_file: 
   blast_results = result_handle.read() 
   save_file.write(blast_results)

In [ ]:
import os

cmd = "blastn -db alun -query search.fsa -out results_aluuuuuuuu.xml -outfmt 5"
os.system(cmd)

In [ ]:
from Bio.Blast import NCBIXML
E_VALUE_THRESH = 1e-20 
for alignment in NCBIXML.parse(open("blast_input.xml")):
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print("****Alignment****")
            print("sequence:", alignment.title)
            print("length:", alignment.length)
            print("e value:", hsp.expect)
            print(hsp.query[0:100] + "...")
            print(hsp.match[0:100] + "...")
            print(hsp.sbjct[0:100] + "...")

In [25]:
############################################# ALU FINDER ####################################################################
###### MAKE LOCAL DATABASE ######
from Bio.Blast.Applications import NcbimakeblastdbCommandline
db_fasta_raw_filename = "13396_Alu_Elements.fsa"
local_dbname = "local_alu_db"
print("Creating Local Database ...")

makeblastdb_cline = NcbimakeblastdbCommandline(dbtype="nucl", input_file= db_fasta_raw_filename ,out = local_dbname )
print("CMD : " , makeblastdb_cline)
stdout,stderr = makeblastdb_cline()

Creating Local Database ...
CMD :  makeblastdb -out local_alu_db -dbtype nucl -in 13396_Alu_Elements.fsa


In [26]:
from Bio.Blast.Applications import NcbiblastnCommandline
import subprocess

seq = "blast_input" 

print("0 = pairwise")
print("1 = query-anchored showing identities")
print("2 = query-anchored no identities")
print("3 = flat query-anchored, show identities")
print("4 = flat query-anchored, no identities")
print("5 = XML Blast output")
print("6 = tabular")
print("7 = tabular with comment lines")
print("8 = Text ASN.1")
print("9 = Binary ASN.1")
print("10 = Comma-separated values")

# command_line = ['blastn','-query',
#                seq + '.fasta','-out',
#                seq + '_blout', '-outfmt',
#                '6','-db','local_alu_db']

command_line = "blastn -db local_alu_db -query " + seq + ".fasta " +"-out " + seq + ".xml -outfmt 5 "


print("\n Result :")
subprocess.call(command_line)



0 = pairwise
1 = query-anchored showing identities
2 = query-anchored no identities
3 = flat query-anchored, show identities
4 = flat query-anchored, no identities
5 = XML Blast output
6 = tabular
7 = tabular with comment lines
8 = Text ASN.1
9 = Binary ASN.1
10 = Comma-separated values

 Result :


0

In [27]:
def parse_blast(resultfile): #takes in the BLAST result, outputs list that can be made into csv rows
    from Bio.Blast import NCBIXML
    result_handle = open(resultfile)
    blast_records = NCBIXML.parse(result_handle)
    csv_list = []
    
    header = [  'Query',
                'Name', 'Length', 'Score', 'Expect',
                'QueryStart', 'QueryEnd',
                'SubjectStart', 'SubjectEnd'
            ]
    
    csv_list.append(header)
    count = 0
    for blast_record in blast_records:
        '''help(blast_record.alignments[0].hsps[0])''' # these give help info for the parts 
        '''help(blast_record.alignments[0])        '''
        count +=1
        
        query = blast_record.query
        for alignment in blast_record.alignments:

            name = alignment.title
            length = alignment.length
    
            hsp = alignment.hsps[0] # I don't know if we will ever have more than one, so might as well take the first one.
            score = hsp.score
            expect = hsp.expect
            querystart = hsp.query_start
            queryend = hsp.query_end
            subjectstart = hsp.sbjct_start
            subjectend = hsp.sbjct_end
            row = [query,name,length,score,expect,querystart,queryend,subjectstart,subjectend]
            csv_list.append(row)
            
    result_handle.close()
    return csv_list

In [28]:
import pandas as pd
import csv
blast_ot =  "blast_input.xml"
output_csv = parse_blast(blast_ot)
filename = "output.csv"
with open(filename, 'w') as csvfile: 
    csvwriter = csv.writer(csvfile)
    for line in output_csv:
        print(line)
        csvwriter.writerow(line)


['Query', 'Name', 'Length', 'Score', 'Expect', 'QueryStart', 'QueryEnd', 'SubjectStart', 'SubjectEnd']
['Example 1', 'gnl|BL_ORD_ID|5 1:006820988:fw:REF-plus:GGCCGGGCGC:305', 317, 116.0, 6.56851e-58, 341, 625, 2, 296]
['Example 1', 'gnl|BL_ORD_ID|37 1:028673658:fw:REF-plus:GGCCGGGCGC:338', 350, 106.0, 2.37923e-52, 341, 629, 2, 296]
['Example 1', 'gnl|BL_ORD_ID|48 1:036261192:fw:REF-plus:GGCCGGGCGC:323', 335, 87.0, 8.67916e-42, 351, 627, 12, 298]
['Example 1', 'gnl|BL_ORD_ID|6 1:006953058:fw:REF-plus:GGCCGGGCGC:291', 303, 82.0, 5.22351e-39, 341, 530, 2, 192]
['Example 1', 'gnl|BL_ORD_ID|11 1:009826434:fw:REF-plus:GGCCGGGCGC:285', 297, 72.0, 1.89205e-33, 31, 306, 6, 287]
['Example 1', 'gnl|BL_ORD_ID|14 1:010614064:fw:REF-plus:GGCCGGGCGC:326', 338, 63.0, 1.90548e-28, 143, 320, 133, 314]
['Example 1', 'gnl|BL_ORD_ID|133 1:112302314:fw:REF-plus:GGCCGGGCGC:311', 323, 55.0, 5.33556e-24, 93, 318, 71, 297]
['Example 1', 'gnl|BL_ORD_ID|74 1:062390602:fw:REF-plus:GGCCGGGCGC:344', 356, 52.0, 2.482

In [29]:
import pandas as pd 
from Bio import SeqIO

output_csv = pd.read_csv("output.csv")
Alu_start_position_list = output_csv.QueryStart
Alu_end_position_list = output_csv.QueryEnd
alu_locations_df = pd.DataFrame(list(zip(Alu_start_position_list, Alu_end_position_list)),columns =['Start Position', 'End Position'])
seq_record = SeqIO.parse(open('blast_input.fasta'),'fasta')

# Removing Alu Element and Zipping it together

for seq_object in seq_record :
    for i in range(0,alu_locations_df.shape[0]):
        seq = str(seq_object.seq)
        Query_Start = alu_locations_df.at[i, "Start Position"] - 1
        Query_End = alu_locations_df.at[i, "End Position"] - 1
        print("\n Start Position : " , Query_Start , "\n End Position : " , Query_End , "\n Deleted Element : " , seq[Query_Start:Query_End] , "\n")
        seq = seq.replace(seq[Query_Start:Query_End], '')
    print("\n Sequence : " , seq) 


    




 Start Position :  340 
 End Position :  624 
 Deleted Element :  GCCAGGTGTGGTGGCTCACGCCTGTAATCCCACCGCTTTGGGAGGCTGAGTCAGATCACCTGAGGTTAGGAATTTGGGACCAGCCTGGCCAACATGGCGACACCCCAGTCTCTACTAATAACACAAAAAATTAGCCAGGTGTGCTGGTGCATGTCTGTAATCCCAGCTACTCAGGAGGCTGAGGCATGAGAATTGCTCACGAGGCGGAGGTTGTAGTGAGCTGAGATCGTGGCACTGTACTCCAGCCTGGCGACAGAGGGAGAACCCATGTCAAAAACAAAAAA 


 Start Position :  340 
 End Position :  628 
 Deleted Element :  GCCAGGTGTGGTGGCTCACGCCTGTAATCCCACCGCTTTGGGAGGCTGAGTCAGATCACCTGAGGTTAGGAATTTGGGACCAGCCTGGCCAACATGGCGACACCCCAGTCTCTACTAATAACACAAAAAATTAGCCAGGTGTGCTGGTGCATGTCTGTAATCCCAGCTACTCAGGAGGCTGAGGCATGAGAATTGCTCACGAGGCGGAGGTTGTAGTGAGCTGAGATCGTGGCACTGTACTCCAGCCTGGCGACAGAGGGAGAACCCATGTCAAAAACAAAAAAAGAC 


 Start Position :  350 
 End Position :  626 
 Deleted Element :  GTGGCTCACGCCTGTAATCCCACCGCTTTGGGAGGCTGAGTCAGATCACCTGAGGTTAGGAATTTGGGACCAGCCTGGCCAACATGGCGACACCCCAGTCTCTACTAATAACACAAAAAATTAGCCAGGTGTGCTGGTGCATGTCTGTAATCCCAGCTACTCAGGAGGCTGAGGCATGAGAATTGCTCACGAGGCGGAGGTTGTAGTGAGCTGAGATCGTG

In [30]:
import pandas as pd 
from Bio import SeqIO

output_csv = pd.read_csv("output.csv")
Alu_start_position_list = output_csv.QueryStart
Alu_end_position_list = output_csv.QueryEnd
alu_locations_df = pd.DataFrame(list(zip(output_csv.Name, output_csv.QueryStart, output_csv.QueryEnd)),columns =['Name' ,'Start Position', 'End Position'])
seq_record = SeqIO.parse(open('blast_input.fasta'),'fasta')


# Locus Finder

Locus_df = pd.DataFrame(columns = ['Locus Start Position', 'Locus End Position' , 'Locus', 'Query Match'])

for seq_object in seq_record :
    for i in range(0,alu_locations_df.shape[0]):
        seq = str(seq_object.seq)
        Query_Start = alu_locations_df.at[i, "Start Position"] - 1
        Query_End = alu_locations_df.at[i, "End Position"] - 1
        Query_Match = alu_locations_df.at[i, "Name"]
        Locus_Downsteam = seq[max(0,Query_Start-100):Query_Start]
        Locus_Upstream = seq[Query_End + 1:min(Query_End + 100 , len(seq))]
        #print("\n Start Position : " , Query_Start , "\n End Position : " , Query_End ,"\n Match : " , Query_Match ,"\n Transposon : " , seq[Query_Start:Query_End] , "\n")
        Locus = Locus_Downsteam + Locus_Upstream
        #print("\n Locus : " , Locus)
        Locus_df = Locus_df.append({'Locus Start Position' : max(0,Query_Start-100), 'Locus End Position' : min(Query_End + 100 , len(seq)), 'Locus' : Locus, 'Query Match' : Query_Match}, ignore_index = True)

Locus_df.to_csv('LocusIdentifier_Output.csv', index = True)

C:\Users\papac\AppData\Local\Temp\ipykernel_5588\1007790438.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Locus_df = Locus_df.append({'Locus Start Position' : max(0,Query_Start-100), 'Locus End Position' : min(Query_End + 100 , len(seq)), 'Locus' : Locus, 'Query Match' : Query_Match}, ignore_index = True)
C:\Users\papac\AppData\Local\Temp\ipykernel_5588\1007790438.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Locus_df = Locus_df.append({'Locus Start Position' : max(0,Query_Start-100), 'Locus End Position' : min(Query_End + 100 , len(seq)), 'Locus' : Locus, 'Query Match' : Query_Match}, ignore_index = True)
C:\Users\papac\AppData\Local\Temp\ipykernel_5588\1007790438.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat ins

In [31]:
Str = "0123456789"
print(Str[5:9+100])

56789
